In [1]:
import kagglehub

c:\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = kagglehub.dataset_download("kshitij192/cars-image-dataset")

100%|██████████| 36.3M/36.3M [00:12<00:00, 3.11MB/s]

Extracting files...
